In [26]:
import openai, pandas as pd

In [27]:
import openai
import os
from dotenv import load_dotenv

# Load environment variables from your custom file
load_dotenv(dotenv_path='agent.env')

# Get the OpenAI API key from the loaded environment
openai.api_key = os.getenv("OPENAI_API_KEY")


In [28]:
print("API Key loaded:", openai.api_key[:5] + "..." if openai.api_key else "Not found")


API Key loaded: sk-pr...


In [29]:
# print(client.organization)


In [30]:
import pymysql
import pandas as pd
from openai import OpenAI
import re
from dotenv import load_dotenv
import os

# Load secrets
load_dotenv("agent.env")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_mysql_connection():
    return pymysql.connect(
        host=os.getenv("MYSQL_HOST"),
        port=int(os.getenv("MYSQL_PORT")),
        user=os.getenv("MYSQL_USER"),
        password=os.getenv("MYSQL_PASSWORD"),
        database=os.getenv("MYSQL_DATABASE"),
        connect_timeout=5
    )

def ask_sql_question(user_question, schema_context):
    try:
        # Step 1: Generate SQL from natural language
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes SQL queries for a MySQL database."},
                {"role": "user", "content": f"{schema_context}\nConvert this request into SQL: {user_question}"}
            ]
        )

        full_response = response.choices[0].message.content.strip()

        # Extract SQL code from response
        match = re.search(r"```sql\n(.*?)```", full_response, re.DOTALL)
        generated_sql = match.group(1).strip() if match else full_response

        print("🧠 Generated SQL:\n", generated_sql)

        # Step 2: Execute the query
        conn = get_mysql_connection()
        cursor = conn.cursor()
        cursor.execute(generated_sql)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        cursor.close()
        conn.close()

        # Step 3: Return DataFrame
        return pd.DataFrame(rows, columns=columns)

    except Exception as e:
        print("❌ Error:", e)
        return None


In [31]:
schema_context = """
You are working with a MySQL database called 'us_project' that contains the following tables:

us_household_income:
  - row_id (int)
  - id (int)
  - State_Code (int)
  - State_Name (text)
  - State_ab (text)
  - County (text)
  - City (text)
  - Place (text)
  - Type (text)
  - Primary (text)
  - Zip_Code (int)
  - Area_Code (int)
  - ALand (int)
  - AWater (int)
  - Lat (double)
  - Lon (double)


us_household_income_statistics:
  - id (int)
  - State_Name (text)
  - Mean (int)
  - Median (int)
  - Stdev (int)
  - sum_w (double)

water_content_view:
  - State_Name (text)
  - City (text)
  - County (text)
  - ALand (int)
  - AWater (int)
  - Total_area (bigint)
  - WaterPercent (decimal)
  - Water_content (varchar)

"""

ask_sql_question("Which states have the highest median income?", schema_context)


🧠 Generated SQL:
 SELECT State_Name
FROM us_household_income_statistics
ORDER BY Median DESC
LIMIT 1;


,State_Name
0,Alabama


In [32]:
conn = get_mysql_connection()
cursor = conn.cursor()

# Get all tables
cursor.execute("SHOW TABLES;")
tables = [t[0] for t in cursor.fetchall()]

schema_lines = []
for table in tables:
    schema_lines.append(f"{table}:")
    cursor.execute(f"""
        SELECT COLUMN_NAME, DATA_TYPE 
        FROM INFORMATION_SCHEMA.COLUMNS 
        WHERE TABLE_SCHEMA = '{os.getenv("MYSQL_DATABASE")}' AND TABLE_NAME = '{table}';
    """)
    for column, dtype in cursor.fetchall():
        schema_lines.append(f"  - {column} ({dtype})")
    schema_lines.append("")

cursor.close()
conn.close()

schema_context = "\n".join(schema_lines)
print(schema_context)



us_household_income:
  - row_id (int)
  - id (int)
  - State_Code (int)
  - State_Name (text)
  - State_ab (text)
  - County (text)
  - City (text)
  - Place (text)
  - Type (text)
  - Primary (text)
  - Zip_Code (int)
  - Area_Code (int)
  - ALand (int)
  - AWater (int)
  - Lat (double)
  - Lon (double)

us_household_income_statistics:
  - id (int)
  - State_Name (text)
  - Mean (int)
  - Median (int)
  - Stdev (int)
  - sum_w (double)

water_content_view:
  - State_Name (text)
  - City (text)
  - County (text)
  - ALand (int)
  - AWater (int)
  - Total_area (bigint)
  - WaterPercent (decimal)
  - Water_content (varchar)



In [33]:
ask_sql_question("Group by States and County, and only states up to letter 'M'?", schema_context)

🧠 Generated SQL:
 SELECT State_Name, County, 
       SUM(ALand) AS Total_Area_Land, 
       SUM(AWater) AS Total_Area_Water, 
       SUM(ALand + AWater) AS Total_Area,
       ROUND(SUM(AWater) / SUM(ALand + AWater) * 100, 2) AS Water_Percentage
FROM water_content_view
WHERE State_Name < 'N'
GROUP BY State_Name, County


,State_Name,County,Total_Area_Land,Total_Area_Water,Total_Area,Water_Percentage
0,Alabama,Autauga County,54876796422,1832810687,56709607109,3.23
1,Alabama,Chambers County,43254712,82933,43337645,0.19
2,Alabama,Pickens County,17151290,59271,17210561,0.34
3,Alabama,Jefferson County,193512363,1424126,194936489,0.73
4,Alabama,Lee County,396756288,5429121,402185409,1.35
...,...,...,...,...,...,...
853,Montana,Rosebud County,1979114,24971,2004085,1.25
854,Montana,Toole County,19575595,1109829,20685424,5.37
855,Montana,Jefferson County,15222898,0,15222898,0.00
856,Montana,Cascade County,52411082,1439605,53850687,2.67
